In [4]:
import pandas as pd

df_ratings = pd.read_csv('C:/Jupyter_project/datasets/movie_lens/rating.csv')
df_movies= pd.read_csv('C:\Jupyter_project\datasets\movie_lens/movie.csv')

In [5]:
#데이터를 줄이기 위한 코드 --여기에 선노님 함수 넣으면 될 것 같습니다
x= df_ratings['userId'].value_counts()>500
y=x[x].index
df_ratings = df_ratings[df_ratings['userId'].isin(y)]

In [6]:
from scipy.sparse.linalg import svds
def make_prediction_df():
    R_df = df_ratings.pivot(index = 'userId', columns ='movieId', values = 'rating').fillna(0)

    #data를 dataframe에서 numpy array로 전환한다.
    R = R_df.values
    user_ratings_mean = np.mean(R, axis = 1)
    R_demeaned = R - user_ratings_mean.reshape(-1, 1)
        
    U, sigma, Vt = svds(R_demeaned, k = 50)
    
    sigma = np.diag(sigma)
    all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1,1)
    
    preds_df = pd.DataFrame(all_user_predicted_ratings, columns = R_df.columns)
    
    return preds_df


In [7]:
def recommend_movies(predictions_df, userID, movies_df, original_ratings_df, num_recommendations=5):
    
    user_row_number = userID - 1
    sorted_user_predictions = predictions_df.iloc[user_row_number].sort_values(ascending=False)
    
    user_data = original_ratings_df[original_ratings_df.userId == (userID)]
    user_full = (user_data.merge(movies_df, how = 'left', left_on = 'movieId', right_on = 'movieId').
                     sort_values(['rating'], ascending=False)
                 )

    recommendations = (movies_df[~movies_df['movieId'].isin(user_full['movieId'])].
         merge(pd.DataFrame(sorted_user_predictions).reset_index(), how = 'left',
               left_on = 'movieId',
               right_on = 'movieId').
         rename(columns = {user_row_number: 'Predictions'}).
         sort_values('Predictions', ascending = False).
                       iloc[:num_recommendations, :-1]
                      )

    return user_full


In [10]:
#make_prediction_df()함수 호출하기 전에 input을 넣어줍니다.
preds_df = make_prediction_df()

In [11]:
already_rated = recommend_movies(preds_df,24, df_movies, df_ratings, 10)

In [12]:
already_rated.head(10)

,userId,movieId,rating,timestamp,title,genres
170,24,1594,5.0,2001-07-04 07:50:13,In the Company of Men (1997),Comedy|Drama
383,24,3785,5.0,2001-10-11 10:38:05,Scary Movie (2000),Comedy|Horror
128,24,1214,5.0,2001-07-01 07:23:08,Alien (1979),Horror|Sci-Fi
366,24,3578,5.0,2001-07-01 07:54:22,Gladiator (2000),Action|Adventure|Drama
70,24,457,5.0,2001-07-04 07:03:07,"Fugitive, The (1993)",Thriller
370,24,3617,5.0,2001-07-01 07:56:54,Road Trip (2000),Comedy
130,24,1222,5.0,2001-09-10 05:13:38,Full Metal Jacket (1987),Drama|War
131,24,1233,5.0,2001-07-02 10:52:17,"Boot, Das (Boat, The) (1981)",Action|Drama|War
132,24,1240,5.0,2001-07-01 07:23:50,"Terminator, The (1984)",Action|Sci-Fi|Thriller
133,24,1258,5.0,2001-09-10 05:13:38,"Shining, The (1980)",Horror
